In [5]:
#!pip install SpeechRecognition
#!brew install portaudio
!pip install pyaudio
!pip install websocket-client

/bin/sh: brew: command not found
  Using cached https://files.pythonhosted.org/packages/ab/42/b4f04721c5c5bfc196ce156b3c768998ef8c0ae3654ed29ea5020c749a6b/PyAudio-0.2.11.tar.gz
  Complete output from command /home/nbuser/anaconda3_420/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-0pjuoefn/pyaudio/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /tmp/pip-wheel-wlgp3vjk --python-tag cp35:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.5
  copying src/pyaudio.py -> build/lib.linux-x86_64-3.5
  running build_ext
  building '_portaudio' extension
  creating build/temp.linux-x86_64-3.5
  creating build/temp.linux-x86_64-3.5/src
  gcc -pthread -B /home/nbuser/anaconda3_420/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/nbuser/anaconda3_42

In [7]:
from threading import Thread
import os, io, struct, uuid, wave, sys, pyaudio, requests, urllib.parse, websocket, _thread

def get_wave_header(frame_rate):
    if frame_rate not in [8000, 16000]:
        raise ValueError('Frame rate has to be 8000 or 16000')
        
    nchannels = 1
    bytes_per_sample = 2
    
    header = io.BytesIO()
    header.write(str.encode('RIFF'))
    header.write(struct.pack('<L', 0))
    header.write(str.encode('WAVE'))
    header.write(str.encode('fmt'))
    header.write(struct.pack('<L', 16))
    header.write(struct.pack('<H', 1))
    header.write(struct.pack('<H', nchannels))
    header.write(struct.pack('<L', frame_rate))
    header.write(struct.pack('<L', frame_rate * nchannels * bytes_per_sample))
    header.write(struct.pack('<H', nchannels * bytes_per_sample))
    header.write(struct.pack('<H', bytes_per_sample * 8))
    header.write(str.encode('data'))
    header.write(struct.pack('<L', 0))
    
    data = header.getvalue()
    header.close()
    return data


ImportError: No module named 'pyaudio'

In [5]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

params = urllib.parse.urlencode(
    {
        'from':'en-US',
        'to':'es-MX',
        'features':'Partial,TextToSpeech,TimingInfo',
        'voice':'es-MX-Sabina',
        'api-version':'1.0'
    }
)

output_folder = os.path.join(os.getcwd(), uuid.uuid4().hex)

tts_state = {'count':0}

def on_open(ws):
    print('Opened')
    
    def run(*args):
        try:
            print('Running')
            data = get_wave_header(16000)
            ws.send(data, websocket.ABNF.OPCODE_BINARY)

            while True:
            #for i in range(0, RATE // CHUNK * RECORD_SECONDS):
                sys.stdout.write('.')
                data = stream.read(CHUNK, exception_on_overflow=False)
                ws.send(data, websocket.ABNF.OPCODE_BINARY)

            stream.stop_stream()
            stream.close()
            p.terminate()
            ws.close()
        except Exception as ex:
            print(ex)
        
    _thread.start_new_thread(run, ())
    
def on_close(ws):
    print('Closed')
    
def on_error(ws, error):
    print(error)
    
def on_data(ws, message, message_type, fin):
    print('Receiving data')
    if message_type == websocket.ABNF.OPCODE_BINARY:
        print(message)
    else:
        tts_count = tts_state['count']
        tts_file = tts_state.get('file', None)
        if tts_file is None:
            tts_count += 1
            tts_state['count'] = tts_count
            file_name = "tts_{0}.wav".format(tts_count)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            tts_file = open(os.path.join(output_folder, file_name), 'wb')
            tts_state['file'] = tts_file
        tts_file.write(message)
        if fin:
            tts_file.close()
            del tts_state['file']
            os.system('aplay ' + output_folder + '/' + file_name)

client_trace_id = str(uuid.uuid4())
request_url = 'wss://dev.microsofttranslator.com/speech/translate?%s' % params
headers = {
    'Ocp-Apim-Subscription-Key':'696c0961cc074369badc0544ca90bcf2',
    'X-ClientTraceId':client_trace_id
}

ws_client = websocket.WebSocketApp(request_url, headers, on_open=on_open, on_data=on_data, on_error=on_error, on_close=on_close)
ws_client.run_forever()

Opened
Running
........Closed
Connection is already closed.
